In [16]:
import os
import json
import torch
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from torchvision import transforms
from ultralytics import YOLO
import time 
import numpy as np
from torchvision.ops import box_iou

In [2]:
class TrafficSignDataset(Dataset):
    def __init__(self, image_dir, label_dir, class_map, transforms=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transforms = transforms
        self.class_map = class_map
        self.image_files = sorted(os.listdir(image_dir))

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, os.path.splitext(img_name)[0] + '.json')

        # Wymuszenie RGB i konwersja do tensora
        img = Image.open(img_path).convert("RGB")
        image = transforms.ToTensor()(img)  # ToTensor dzieli przez 255 automatycznie

        with open(label_path, 'r') as f:
            data = json.load(f)[0]
            annotations = data["annotations"]

        boxes = []
        labels = []

        for ann in annotations:
            label = ann["label"]
            coords = ann["coordinates"]
            x1 = coords["x"] - coords["width"] / 2
            y1 = coords["y"] - coords["height"] / 2
            x2 = coords["x"] + coords["width"] / 2
            y2 = coords["y"] + coords["height"] / 2

            boxes.append([x1, y1, x2, y2])
            labels.append(self.class_map[label])

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64),
        }

        return image, target

    def __len__(self):
        return len(self.image_files)
    
def load_class_map(filepath):
    with open(filepath, "r") as f:
        lines = f.read().splitlines()
    return {label: idx + 1 for idx, label in enumerate(lines)}

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [3]:
def load_class_map(path):
    with open(path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    return {cls: idx+1 for idx, cls in enumerate(classes)}  # +1, bo 0 to background

def get_model(num_classes, weights: FasterRCNN_ResNet50_FPN_Weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=weights)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

def collate_fn(batch):
    return tuple(zip(*batch))

def train(model):
    if model == "rcnn":
        start_time = time.time()
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        label_map = load_class_map("detection/labels/classes.txt")

        dataset = TrafficSignDataset(
            image_dir="detection/images/train",
            label_dir="detection/labels/train",
            class_map=label_map
        )

        data_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

        model = get_model(num_classes=len(label_map) + 1)
        model.to(device)

        optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

        num_epochs = 10
        for epoch in range(num_epochs):
            model.train()
            total_loss = 0
            progress_bar = tqdm(data_loader, desc=f"Epoch {epoch+1}", leave=True)
            for i, (images, targets) in enumerate(tqdm(data_loader, desc=f"Epoch {epoch+1}")):
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())

                optimizer.zero_grad()
                losses.backward()
                optimizer.step()
                total_loss += losses.item()
                progress_bar.set_postfix(loss=losses.item())

            print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")

        torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pth")
        elapsed_time = time.time() - start_time
        print(f"[RCNN] Training completed in {elapsed_time:.2f} seconds.")
    elif model =="yolo":
        print("Training YOLOv8 model...")
        start_time1 = time.time()
        project_dir = r"C:\Users\zuzan\Desktop\rz"

        yaml_path = os.path.join(project_dir, "dataset.yaml")

        class_names = [
            'A-1','A-11','A-11a','A-12a','A-14','A-15','A-16','A-17','A-18b',
            'A-2','A-20','A-21','A-24','A-29','A-3','A-30','A-32','A-4','A-6a',
            'A-6b','A-6c','A-6d','A-6e','A-7','A-8','B-1','B-18','B-2',
            'B-20','B-21','B-22','B-25','B-26','B-27','B-33','B-34','B-36',
            'B-41','B-42','B-43','B-44','B-5','B-6-C-8-C-9','B-8','B-9','C-10','C-12',
            'C-13','C-13-C-16','C-13a','C-13a-C-16a','C-16','C-2','C-4','C-5','C-6',
            'C-7','C-9','D-1','D-14','D-15','D-18','D-18b','D-2','D-21','D-23',
            'D-23a','D-24','D-26','D-26b','D-26c','D-27','D-28','D-29','D-3',
            'D-40','D-41','D-42','D-43','D-4a','D-4b','D-51','D-52','D-53','D-6',
            'D-6b','D-7','D-8','D-9','D-','G-1a','G-3'
        ]

        yaml_content = f"""\
        path: {os.path.join(project_dir, 'detection')}
        train: images/train
        val: images/test
        test: images/test
        nc: {len(class_names)}
        names: {class_names}
        """

        with open(yaml_path, "w") as f:
            f.write(yaml_content)

        print(f"Plik zapisany: {yaml_path}")

        model = YOLO("yolov8n.pt") 

        results = model.train(
            data=yaml_path,
            epochs=100,
            imgsz=640,
            device='0',  # 'cpu' jeśli nie masz GPU
            patience=10,  # Wczesne zatrzymanie jeśli brak poprawy
            lr0=0.01,
            workers=0
        )
        elapsed_time1 = time.time() - start_time1
        print(f"[YOLO] Training completed in {elapsed_time1:.2f} seconds.")
    else:
        raise ValueError("Nieznany typ modelu. Wybierz 'rcnn' lub 'yolo'.")


In [4]:
train(model="yolo")

Training YOLOv8 model...
Plik zapisany: C:\Users\zuzan\Desktop\rz\dataset.yaml


100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 11.7MB/s]


Ultralytics 8.3.156  Python-3.11.5 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\zuzan\Desktop\rz\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plo

100%|█████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:00<00:00, 9.63MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.10.0 ms, read: 2322.1419.7 MB/s, size: 1571.1 KB)


...

     54/100      2.38G      1.181      1.648     0.9077         77        640: 100%|██████████| 24/24 [00:13<00:00,  1.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<0

                   all         78        122      0.579      0.355      0.414      0.304
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 44, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



54 epochs completed in 0.232 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.8MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.8MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.5 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,262,316 parameters, 0 gradients, 9.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<0


                   all         78        122      0.578       0.36      0.424      0.317
                 A-11a          6          6          1      0.648      0.806      0.668
                  A-14          1          1          1          0          0          0
                  A-16          1          1          1          0      0.012     0.0108
                  A-17          1          1          0          0      0.111     0.0663
                   A-2          2          2          0          0          0          0
                  A-20          1          1          0          0          0          0
                  A-21          1          1          0          0     0.0433      0.026
                  A-6c          1          1          0          0          0          0
                  A-6d          1          1          0          0          0          0
                   A-7          2          2      0.778          1      0.995      0.771
                   B-

In [ ]:
train(model="rcnn")

Training completed in 8799.64 seconds.

In [7]:
def load_class_map(path):
    with open(path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    return {cls: idx+1 for idx, cls in enumerate(classes)}

def get_model(num_classes, model_path):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    model.load_state_dict(torch.load(model_path))
    return model

def evaluate_model(model, data_loader, device, iou_threshold=0.5, score_threshold=0.5):
    model.eval()
    all_true_positives = 0
    all_false_positives = 0
    all_false_negatives = 0
    all_true_negatives = 0  
    with torch.no_grad():
        for images, targets in data_loader:
            images = [img.to(device) for img in images]
            outputs = model(images)

            for output, target in zip(outputs, targets):
                pred_boxes = output['boxes'][output['scores'] > score_threshold]
                pred_labels = output['labels'][output['scores'] > score_threshold]

                true_boxes = target['boxes'].to(device)
                true_labels = target['labels'].to(device)

                if len(pred_boxes) == 0:
                    all_false_negatives += len(true_boxes)
                    continue

                ious = box_iou(pred_boxes, true_boxes)

                tp = 0
                matched_true = set()
                for pred_idx in range(ious.size(0)):
                    candidate_true_indices = [i for i in range(ious.size(1))
                                              if ious[pred_idx, i] > iou_threshold and pred_labels[pred_idx] == true_labels[i]]
                    if candidate_true_indices:
                        for true_idx in candidate_true_indices:
                            if true_idx not in matched_true:
                                matched_true.add(true_idx)
                                tp += 1
                                break

                fp = len(pred_boxes) - tp
                fn = len(true_boxes) - len(matched_true)

                all_true_positives += tp
                all_false_positives += fp
                all_false_negatives += fn

    precision = all_true_positives / (all_true_positives + all_false_positives + 1e-6)
    recall = all_true_positives / (all_true_positives + all_false_negatives + 1e-6)
    return precision, recall , all_true_positives, all_false_positives, all_false_negatives

def collate_fn(batch):
    return tuple(zip(*batch))


def results(model):
    if model == 'rcnn':

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        label_map = load_class_map("detection/labels/classes.txt")
        num_classes = len(label_map) + 1

        model_path = "model_epoch_10.pth"

        dataset_test = TrafficSignDataset(
            image_dir="detection/images/test",
            label_dir="detection/labels/test",
            class_map=label_map
        )

        data_loader_test = DataLoader(dataset_test, batch_size=4, shuffle=False, collate_fn=collate_fn)

        model = get_model(num_classes, model_path)
        model.to(device)

        precision, recall, tp, fp, fn = evaluate_model(model, data_loader_test, device)

        os.makedirs("./evaluate_model", exist_ok=True)
        with open("./evaluate_model/evaluate_rcnn.txt", "w") as f:
            f.write(f"Precision: {precision:.4f}\n")
            f.write(f"Recall: {recall:.4f}\n")
            f.write(f"TP: {tp:.4f}\n")
            f.write(f"FP: {fp:.4f}\n")
            f.write(f"FN: {fn:.4f}\n")

        print("RCNN Evaluation complete. Results saved to ./evaluate_model/evaluate_rcnn.txt")

    elif model == 'yolo':
        print("Evaluating YOLOv8 model...")

        model = YOLO("runs/detect/train/weights/best.pt")
        metrics = model.val()

        os.makedirs("./evaluate_model", exist_ok=True)
        with open("./evaluate_model/evaluate_yolo.txt", "w") as f:
            f.write(f"Precision: {np.mean(metrics.box.p):.4f}\n")     
            f.write(f"Recall: {np.mean(metrics.box.r):.4f}\n")        
            f.write(f"mAP@0.5: {metrics.box.map50:.4f}\n")
            f.write(f"mAP@0.5:0.95: {metrics.box.map:.4f}\n")
            
        print("YOLO Evaluation complete. Results saved to ./evaluate_model/evaluate_yolo.txt")

    else:
        raise ValueError("Nieznany model_type. Wybierz 'rcnn' lub 'yolo'.")

In [8]:
results(model="yolo")

Evaluating YOLOv8 model...
Ultralytics 8.3.156  Python-3.11.5 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 3,262,316 parameters, 0 gradients, 9.2 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 2234.31082.2 MB/s, size: 429.7 KB)


val: Scanning C:\Users\zuzan\Desktop\rz\detection\labels\test.cache... 78 images, 0 backgrounds, 0 corrupt: 100%|██████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<0


                   all         78        122      0.592      0.348      0.424      0.316
                 A-11a          6          6          1      0.579      0.814      0.673
                  A-14          1          1          1          0          0          0
                  A-16          1          1          1          0     0.0126     0.0126
                  A-17          1          1          0          0      0.124     0.0746
                   A-2          2          2          0          0          0          0
                  A-20          1          1          0          0          0          0
                  A-21          1          1          0          0     0.0433      0.026
                  A-6c          1          1          0          0          0          0
                  A-6d          1          1          0          0          0          0
                   A-7          2          2      0.819          1      0.995      0.771
                   B-

In [17]:
results(model='rcnn')

RCNN Evaluation complete. Results saved to ./evaluate_model/evaluate_rcnn.txt


In [24]:
def detect_from_image(model_type, image_path, model_path, class_map_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Załaduj etykiety
    label_map = load_class_map(class_map_path)
    rev_label_map = {v: k for k, v in label_map.items()}

    # Wczytaj obraz
    image_bgr = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    image_tensor = torch.from_numpy(image_rgb).permute(2, 0, 1).float() / 255.0
    image_tensor = image_tensor.unsqueeze(0).to(device)

    if model_type == "rcnn":
        # === RCNN ===
        model = get_model(len(label_map) + 1, model_path)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval().to(device)

        with torch.no_grad():
            prediction = model(image_tensor)[0]

        for box, label, score in zip(prediction["boxes"], prediction["labels"], prediction["scores"]):
            if score > 0.5:
                x1, y1, x2, y2 = map(int, box)
                class_name = rev_label_map.get(label.item(), "???")
                cv2.rectangle(image_bgr, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image_bgr, f"{class_name} ({score:.2f})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    elif model_type == "yolo":
        
        model = YOLO(model_path)
        results = model(image_path)[0]  

        boxes = results.boxes.xyxy.cpu().numpy()
        scores = results.boxes.conf.cpu().numpy()
        labels = results.boxes.cls.cpu().numpy().astype(int)

        for box, label, score in zip(boxes, labels, scores):
            if score > 0.5:
                x1, y1, x2, y2 = map(int, box)
                class_name = rev_label_map.get(label + 1, "???") 
                cv2.rectangle(image_bgr, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(image_bgr, f"{class_name} ({score:.2f})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    else:
        raise ValueError("model_type must be 'rcnn' or 'yolo'")

    cv2.imshow(f"{model_type.upper()} Detection", image_bgr)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [23]:
detect_from_image(
        model_type="yolo",
        image_path="detection/images/test/Screenshot_27.png",
        model_path="runs/detect/train/weights/best.pt",
        class_map_path="detection/labels/classes.txt"
    )


image 1/1 C:\Users\zuzan\Desktop\rz\detection\images\test\Screenshot_27.png: 320x640 1 B-2, 1 B-36, 1 C-5, 1 D-3, 40.6ms
Speed: 2.4ms preprocess, 40.6ms inference, 2.7ms postprocess per image at shape (1, 3, 320, 640)


In [25]:
detect_from_image(
        model_type="rcnn",
        image_path="detection/images/test/Screenshot_27.png",
        model_path="model_epoch_10.pth",
        class_map_path="detection/labels/classes.txt"
    )

C:\Users\zuzan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\zuzan\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [28]:
def detect_from_video(model_type, video_path, model_path, class_map_path, output_path="output/result.mp4", show=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    label_map = load_class_map(class_map_path)
    rev_label_map = {v: k for k, v in label_map.items()}

    if model_type == "rcnn":
        model = get_model(len(label_map) + 1, model_path)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval().to(device)
    elif model_type == "yolo":
        model = YOLO(model_path)
    else:
        raise ValueError("model_type must be 'rcnn' or 'yolo'")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Błąd przy otwieraniu pliku wideo.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if model_type == "rcnn":
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_tensor = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
            img_tensor = img_tensor.unsqueeze(0).to(device)

            with torch.no_grad():
                prediction = model(img_tensor)[0]

            for box, label, score in zip(prediction["boxes"], prediction["labels"], prediction["scores"]):
                if score > 0.7:
                    x1, y1, x2, y2 = map(int, box.tolist())
                    label_text = rev_label_map.get(label.item(), "???")
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"{label_text} ({score:.2f})", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        elif model_type == "yolo":
            results = model(frame)[0]
            boxes = results.boxes.xyxy.cpu().numpy()
            scores = results.boxes.conf.cpu().numpy()
            labels = results.boxes.cls.cpu().numpy().astype(int)

            for box, label, score in zip(boxes, labels, scores):
                if score > 0.7:
                    x1, y1, x2, y2 = map(int, box)
                    label_text = rev_label_map.get(label + 1, "???")  # YOLO labels are 0-based
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, f"{label_text} ({score:.2f})", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        out.write(frame)
        if show:
            cv2.imshow("Detection", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

    cap.release()
    out.release()
    if show:
        cv2.destroyAllWindows()

In [30]:
detect_from_video(
    model_type="yolo",
    video_path="video/video_Torun.mp4",
    model_path="runs/detect/train/weights/best.pt",
    class_map_path="detection/labels/classes.txt",
    output_path="output/result_yolo.mp4"
)


0: 384x640 1 D-1, 43.0ms
Speed: 7.0ms preprocess, 43.0ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 40.1ms
Speed: 3.9ms preprocess, 40.1ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 39.8ms
Speed: 3.3ms preprocess, 39.8ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 20.4ms
Speed: 3.1ms preprocess, 20.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 20.4ms
Speed: 2.4ms preprocess, 20.4ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 20.5ms
Speed: 2.3ms preprocess, 20.5ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 20.7ms
Speed: 2.3ms preprocess, 20.7ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 20.4ms
Speed: 2.0ms preprocess, 20.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-


0: 384x640 1 D-6, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 7.3ms
Speed: 1.1ms preprocess, 7.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 1.1ms preprocess, 7.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 1.1ms preprocess, 7.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 8.0ms
Speed: 1.2ms preprocess, 8.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 7.4ms
Speed: 1.2ms preprocess, 7.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 7.0ms
Speed: 0.9ms preprocess, 7.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0

Speed: 1.3ms preprocess, 9.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 9.4ms
Speed: 1.2ms preprocess, 9.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 9.8ms
Speed: 1.2ms preprocess, 9.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 C-9, 1 D-6, 1 D-6b, 9.9ms
Speed: 1.4ms preprocess, 9.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 9.5ms
Speed: 1.2ms preprocess, 9.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 1 D-6b, 9.6ms
Speed: 1.3ms preprocess, 9.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 9.9ms
Speed: 1.3ms preprocess, 9.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 1 D-6b, 9.7ms
Speed: 1.4ms preprocess, 9.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-6, 9.7ms
S


0: 384x640 (no detections), 8.4ms
Speed: 1.3ms preprocess, 8.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.2ms
Speed: 1.2ms preprocess, 8.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.1ms
Speed: 1.1ms preprocess, 8.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.8ms
Speed: 1.1ms preprocess, 7.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.6ms
Speed: 1.3ms preprocess, 8.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.7ms
Speed: 1.2ms preprocess, 9.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.9ms
Speed: 1.6ms preprocess, 11.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.7ms
Speed: 1.1ms preprocess, 7.7ms inference, 0.8m


0: 384x640 1 B-36, 7.3ms
Speed: 1.1ms preprocess, 7.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 7.3ms
Speed: 1.3ms preprocess, 7.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 7.4ms
Speed: 1.1ms preprocess, 7.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 7.3ms
Speed: 1.4ms preprocess, 7.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 7.3ms
Speed: 1.1ms preprocess, 7.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 7.3ms
Speed: 1.1ms preprocess, 7.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 7.6ms
Speed: 1.1ms preprocess, 7.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 7.3ms
Speed: 1.3ms preprocess, 7.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 7.3m

...


0: 384x640 1 D-15, 9.3ms
Speed: 1.3ms preprocess, 9.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-15, 9.4ms
Speed: 1.3ms preprocess, 9.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-15, 9.4ms
Speed: 1.3ms preprocess, 9.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-15, 9.8ms
Speed: 1.3ms preprocess, 9.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-15, 9.5ms
Speed: 1.2ms preprocess, 9.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-15, 9.5ms
Speed: 1.3ms preprocess, 9.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-15, 9.4ms
Speed: 1.3ms preprocess, 9.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-15, 9.4ms
Speed: 1.3ms preprocess, 9.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-15, 9.5m

Speed: 1.4ms preprocess, 8.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 9.0ms
Speed: 1.4ms preprocess, 9.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 8.2ms
Speed: 1.5ms preprocess, 8.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.2ms
Speed: 1.3ms preprocess, 8.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 8.8ms
Speed: 1.7ms preprocess, 8.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 8.1ms
Speed: 1.3ms preprocess, 8.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 8.2ms
Speed: 2.2ms preprocess, 8.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 8.2ms
Speed: 1.5ms preprocess, 8.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 8.2ms
Speed: 1.3ms preproces


0: 384x640 2 D-1s, 9.3ms
Speed: 1.4ms preprocess, 9.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 D-1s, 1 D-18, 10.0ms
Speed: 1.4ms preprocess, 10.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 D-1s, 9.5ms
Speed: 1.2ms preprocess, 9.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 D-1s, 9.4ms
Speed: 1.2ms preprocess, 9.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 D-1s, 9.9ms
Speed: 1.3ms preprocess, 9.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 D-1s, 9.6ms
Speed: 1.5ms preprocess, 9.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 9.3ms
Speed: 1.3ms preprocess, 9.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-1, 9.6ms
Speed: 1.4ms preprocess, 9.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 D-


0: 384x640 (no detections), 10.0ms
Speed: 1.3ms preprocess, 10.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.6ms
Speed: 1.7ms preprocess, 9.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.9ms
Speed: 1.5ms preprocess, 9.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.6ms
Speed: 1.8ms preprocess, 9.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.3ms
Speed: 1.2ms preprocess, 9.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.3ms
Speed: 1.4ms preprocess, 9.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.4ms
Speed: 1.6ms preprocess, 9.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.3ms
Speed: 1.3ms preprocess, 9.3ms inference, 0.8m


0: 384x640 1 D-6b, 8.8ms
Speed: 1.2ms preprocess, 8.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 1 D-6b, 8.6ms
Speed: 1.4ms preprocess, 8.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 1 D-6b, 8.3ms
Speed: 1.3ms preprocess, 8.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 1 D-1, 1 D-6b, 8.5ms
Speed: 1.4ms preprocess, 8.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 1 D-1, 1 D-6b, 8.2ms
Speed: 1.3ms preprocess, 8.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-36, 1 D-6b, 8.2ms
Speed: 1.2ms preprocess, 8.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 B-36s, 1 D-6b, 8.6ms
Speed: 1.3ms preprocess, 8.6ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 B-21, 1 B-36, 8.3ms
Speed: 1.3ms preprocess, 8.3ms inference, 2.2ms p

In [31]:
detect_from_video(
    model_type="rcnn",
    video_path="video/video_Torun.mp4",
    model_path="model_epoch_10.pth",
    class_map_path="detection/labels/classes.txt",
    output_path="output/result_rcnn.mp4"
)
